In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

training_model_id = '9a610eef-a6fd-4e2b-bab3-225089abc1a9'

# --- Load artifacts (already a dict) ---
import json
with open(f"{training_model_id}/training_artifacts.json", "r") as f:
    artifacts = json.load(f)

meta = artifacts.get("meta", {})
episodes = artifacts.get("episodes", [])
ep = episodes[0] if episodes else {}

In [ ]:
# --- Build aligned per-tick DataFrame ---
def action_label(a: int) -> str:
    return {-1: "SELL", 0: "HOLD", 1: "BUY"}.get(a, str(a))

def safe_int(x):
    try:
        return int(x)
    except Exception:
        return np.nan

keys = ["timestamps", "actions", "prob_sell", "prob_hold", "prob_buy"]
arrays = [ep.get(k, []) for k in keys]
m = min(map(len, arrays)) if arrays else 0
df = pd.DataFrame({k: ep.get(k, [])[:m] for k in keys})

if not df.empty:
    df["prob_sell"] = pd.to_numeric(df["prob_sell"], errors="coerce")
    df["prob_hold"] = pd.to_numeric(df["prob_hold"], errors="coerce")
    df["prob_buy"] = (1 - (df["prob_sell"] + df["prob_hold"])).clip(0, 1)
    df["action"] = pd.to_numeric(df["actions"], errors="coerce")
    df["action_label"] = df["action"].map(action_label)
    df["t"] = pd.to_numeric(df["timestamps"].map(safe_int), errors="coerce")
    df = df.dropna(subset=["t"]).sort_values("t").reset_index(drop=True)

prices = ep.get("prices", [])
equity = ep.get("equity_curve", [])

In [ ]:
# --- Plotly figures ---

# 1) Probabilities
fig_probs = go.Figure()
fig_probs.add_trace(go.Scatter(x=df["t"], y=df["prob_buy"], mode="lines", name="P(BUY)"))
fig_probs.add_trace(go.Scatter(x=df["t"], y=df["prob_hold"], mode="lines", name="P(HOLD)"))
fig_probs.add_trace(go.Scatter(x=df["t"], y=df["prob_sell"], mode="lines", name="P(SELL)"))
fig_probs.update_layout(title="Class Probabilities over Time", xaxis_title="Tick", yaxis_title="Probability")

# 2) Actions
fig_actions = go.Figure()
for label in ["BUY", "HOLD", "SELL"]:
    sub = df[df["action_label"] == label]
    if not sub.empty:
        fig_actions.add_trace(go.Scatter(
            x=sub["t"], y=sub["action"], mode="markers", name=label,
            text=[f"{label} @ t={t}" for t in sub["t"]]
        ))
fig_actions.update_layout(title="Actions Timeline", xaxis_title="Tick", yaxis_title="Action")

# 3) Equity
fig_equity = go.Figure()
if equity:
    fig_equity.add_trace(go.Scatter(x=list(range(len(equity))), y=equity, mode="lines", name="Equity"))
    fig_equity.update_layout(title="Equity Curve", xaxis_title="Step", yaxis_title="Equity")

# 4) Action counts
counts = df["action_label"].value_counts().reindex(["BUY","HOLD","SELL"]).fillna(0)
fig_counts = go.Figure()
fig_counts.add_trace(go.Bar(x=counts.index.tolist(), y=counts.values.tolist(), name="Action Count"))
fig_counts.update_layout(title="Action Count", xaxis_title="Action", yaxis_title="Count")

# 5) Metadata table
meta_rows = [
    ["Model ID", str(meta.get("model_id"))],
    ["Selection Metric", f"{meta.get('selection_metric')} = {meta.get('selection_value')}"],
    ["Sharpe", str(ep.get("sharpe_ratio"))],
    ["Cum PnL", str(ep.get("cumulative_pnl"))],
    ["Train Range", str(ep.get("train_index_range"))],
    ["Test Range", str(ep.get("test_index_range"))],
]
fig_meta = go.Figure(data=[go.Table(
    header=dict(values=["Key", "Value"]),
    cells=dict(values=[[r[0] for r in meta_rows], [r[1] for r in meta_rows]])
)])
fig_meta.update_layout(title="Run Metadata")

# --- Display in notebook ---
fig_probs.show()
fig_actions.show()
fig_equity.show()
fig_counts.show()
fig_meta.show()
